# 1. Load 1000 Genomes Data

We're just going to work with genotyping data for chromosome 21, which is the [smallest human chromosome](https://en.wikipedia.org/wiki/Chromosome_21). (We've chosen chr21 because it's relatively small -- this should make things a bit faster. Of course, this is a limitation that should not be followed in practice.)



In [25]:
# https://stackoverflow.com/a/44251637/10730311
%env CHR_21_VCF_GZ=/datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

env: CHR_21_VCF_GZ=/datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz


## Filter and convert the chromosome 21 `vcf.gz` file to a simpler matrix format

Regardless of the dimensionality reduction method we'll use, the input is ultimately the same -- a matrix of SNPs and samples.

Here, we use a shell command **that was adapted from PSET 2 part 1** for converting a `vcf.gz` file to a tab-delimited matrix format. This command is contained in [`vcf2tab.sh`](https://github.com/fedarko/plink-182/blob/master/vcf2tab.sh). It's a moderately complex command that does a few things besides just converting this data, so we explain certain parts of it here for the sake of clarity:

### `bcftools query` options

* `-e'AF<0.01 || AF>0.99 || MULTI_ALLELIC=1 || VT!="SNP"'`

  * This excludes SNPs where:

    1. The listed allele frequency is less than 1% or greater than 99% (in either case, the minor allele frequency can be said to be < 1%).

    2. The SNP is tagged with the flag `MULTI_ALLELIC`. We check for this here rather than after running `bcftools query`, because **judging SNPs as multiallelic based only on if any samples have genotypes containing `2` or `3` will silently fail to recognize multiallelic SNPs where all of the samples present only have alleles classified as `0` or `1`.**
    
      * This is demonstrably the case for PSET 2's data: **TODO substantiate claim beyond md5sums being different, like write a python script to go through the vcf and find all lines where the SNP is tagged as `MULTI_ALLELIC` but none of the samples' genotypes at that SNP contain `2` or `3`.**
  
      * These sorts of corner cases are probably pretty rare, especially for datasets like 1000 Genomes with lots of samples (where the odds of at least one sample having a `2` or `3` in its genotype become more and more likely). However, these corner cases do introduce some bias nonetheless, so accounting for them helps ensure that our downstream analyses use the highest-quality data we can provide.
  
    3. The SNP has a `VT` that isn't `SNP`. This means that non-SNP variants (e.g. structural variants, where `VT` is set to `SNV`) will be removed.
    
      * We have to do this because there are some positions listed in the 1000 Genomes data that aren't SNPs -- for example, "`esv3646363;esv3646364`", which represents a copy-number variation. These positions aren't tagged as `MULTI_ALLELIC` even if they contain things like `2`s in their "genotypes", which will break things downstream (not to mention that these positions aren't necessarily directly comparable to SNPs for the purposes of dimensionality reduction -- it seems like an apples-and-oranges thing).

* `-f'[%GT\t]\n'`

  * This option makes `bcftools query` output samples' genotypes for each SNP, where the genotypes are followed by tabs and the SNPs are separated by newlines. (Each genotype is represented as something like `0|1[tab]`; see the "Extracting per-sample tags" section [here](https://samtools.github.io/bcftools/howtos/query.html).)

  * Since this is phased genotyping data, we can safely assume that samples' genotypes are represented with pipes (`0|0`) instead of slashes (`0/0`). See [section 1.4.2 of the VCF spec](https://samtools.github.io/hts-specs/VCFv4.2.pdf) for details.

### `sed` options

* `sed 's/0|0/0/g' | sed 's/0|1/1/g' | sed 's/1|0/1/g' | sed 's/1|1/2/g'`

  * These calls to `sed` convert the genotypes output from `bcftools query` into numbers -- here, this is just the number of minor alleles each sample has (0, 1, or 2).

### `grep` options (not applicable any more)
* PSET 2's version of this command included a `grep -v "|"` call after the `sed` calls, which filtered out any SNP lines still containing a `|` (which would have been the case for any SNPs that contained alleles besides `0` or `1`, e.g. `0|2`, since they wouldn't have matched any of the regular expression queries provided to `sed` above).

* This helped filter *some, but not all* multiallelic SNPs, as described above. Since we now filter out multiallelic SNPs and non-SNP variants "upstream," using `bcftools query`, we can safely remove this line.

In [10]:
%%bash

nohup ~/plink-182/vcf2tab.sh

[W::hts_idx_load2] The index file is older than the data file: /datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi


### For reference, retrieve sample and SNP IDs using `bcftools query`
The sample retrieval command was also taken from PSET 2, part 1.

In [9]:
%%bash

# Get sample IDs
bcftools query -l $CHR_21_VCF_GZ > ~/plink-182/data/sample_ids.txt

# Get SNP IDs (uses the same -e flags as in vcf2tab.sh)
# Ideally these flags would be stored as a string variable or something in a single location, and
# all of these commands would access this string -- but I just spent like 30 minutes trying to make
# that work and bcftools kept ignoring the expression when I passed it that way. So, for the sake of
# my own sanity and because time is short, we're going to do this inelegantly.
bcftools query -e'AF<0.01 || AF>0.99 || MULTI_ALLELIC=1 || VT!="SNP"' -f'%ID\n' $CHR_21_VCF_GZ > ~/plink-182/data/snp_ids.txt

[W::hts_idx_load2] The index file is older than the data file: /datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi


## Get some extra information that will help with simulation/etc.

### SNP Metadata

This is a tab-separated file containing:

- ID
- Location on chr21
- Alternate allele frequency

Along with a header (thanks to the `-H` option of `bcftools query`).

This is essentially just a beefed up version of the "SNP IDs" file we generated above.

In [26]:
%%bash

bcftools query -e'AF<0.01 || AF>0.99 || MULTI_ALLELIC=1 || VT!="SNP"' -H -f'%ID\t%POS\t%INFO/AF\n' $CHR_21_VCF_GZ > ~/plink-182/data/snp_metadata.txt

[W::hts_idx_load2] The index file is older than the data file: /datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi


### Filtered `vcf.gz` file

So that the only SNPs contained are those that we decided to include in the genotyping matrix.

In [28]:
%%bash

# The "--output-type z" makes this produce a vcf.gz file, instead of a plain vcf file (which seems
# to be the default?)
bcftools view -e'AF<0.01 || AF>0.99 || MULTI_ALLELIC=1 || VT!="SNP"' --output-type z $CHR_21_VCF_GZ > ~/plink-182/data/filtered.vcf.gz

[W::hts_idx_load2] The index file is older than the data file: /datasets/cs284s-sp20-public/1000Genomes/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz.tbi


### Sample Metadata
This is a tab-separated file containing:

- Sample ID
- Population
- Superpopulation

A lot of this code was adapted from the `02-Run-Dimensionality-Reduction.ipynb` notebook's visualization code, and that code was in turn adapted from PS2 Part 1's code.

In [41]:
import os
import pandas as pd

PREFIX = os.path.join(os.environ["HOME"], "plink-182")
DATA_PREFIX = os.path.join(PREFIX, "data")

### BEGIN: this code was taken from notebook 2's viz. code, which was in turn taken from PS2 Part 1 ###

sample_to_pop = {}
with open("/datasets/cs284s-sp20-public/ps2/ps2_reference_labels.csv", "r") as f:
    for line in f:
        sample, pop = line.strip().split(",")
        sample_to_pop[sample] = pop

pop_to_superpop = {"None": "None"}
# Derived from the population-label loading code above
with open(os.path.join(DATA_PREFIX, "pop_to_superpop.tsv"), "r") as p2sp:
    for line in p2sp:
        split_line = line.strip().split("\t")
        pop_to_superpop[split_line[0]] = split_line[2]
        
### END: this code was taken from notebook 2's viz. code, which was in turn taken from PS2 Part 1 ###

# (ok, now that we have the requisite dictionaries set up we can map sample IDs to populations painlessly)
sample_metadata = pd.read_csv(
    os.path.join(os.environ["HOME"], "plink-182", "data", "sample_ids.txt"), header=None
)

sample_metadata.columns = ["id"]
sample_metadata["population"] = sample_metadata.apply(lambda row: sample_to_pop.get(row["id"], "None"), axis=1)
sample_metadata["superpopulation"] = sample_metadata.apply(lambda row: pop_to_superpop[row["population"]], axis=1)

sample_metadata.set_index("id", inplace=True)
sample_metadata.to_csv(os.path.join(DATA_PREFIX, "sample_metadata.tsv"), sep="\t")